# Description - Create Swarm File to run transformation to MNI pipeline on the preprocessed data

This notebook contians the code to extract representative timeseries for ROIs using the 7 Networks / 200 ROIs version of the Yeo Atlas.

First, we will prepare the atlas for its use in this work

Next, we will run AFNI's program ```3dNetCorr``` to extract the represenative timeseries. This second step will be done via a swarm job.

Prior to running this notebook you need to make sure the following variables are correctly set:

1. Assign the full path to the atlases folder to variable [```ATLASES_DIR``` in ```basics.py```](https://github.com/nimh-sfim/fc_introspection/blob/main/notebooks/utils/basics.py#L65).

2. Assign the name of the cortical atlas (Schaefer2018_200Parcels_7Networks) to the [```CORTICAL_ATLAS_NAME``` variable in ```basics.py```](https://github.com/nimh-sfim/fc_introspection/blob/main/notebooks/utils/basics.py#L66).

3. Assign the name of the cortical atlas (aal2) to the [```SUBCORTICAL_ATLAS_NAME``` variable in ```basics.py```](https://github.com/nimh-sfim/fc_introspection/blob/main/notebooks/utils/basics.py#L68).

> **NOTE**: Initially, you will only need to change the ATLASES_DIR value to match your system configuration. For the other two, you should be able to use the values provided.


In [1]:
import subprocess
import getpass
import os
import pandas as pd
from datetime import datetime
from shutil import rmtree
from utils.basics import CORTICAL_ATLAS_PATH, CORTICAL_ATLAS_NAME, SUBCORTICAL_ATLAS_PATH, SUBCORTICAL_ATLAS_NAME, FB_ATLAS_NAME, FB_ATLAS_PATH
from utils.basics import DATA_DIR, PRJ_DIR, SCRIPTS_DIR, ATLASES_DIR
from utils.basics import get_sbj_scan_list
import os.path as osp
from sfim_lib.atlases.raking import correct_ranked_atlas

# 1. Atlas Preparation: Schaeffer 200 ROI Atlas

This project uses the 200 ROI version of the Schaefer Atlas sorted according to the 7 Yeo Networks. This atlas is publicly available [here](https://github.com/ThomasYeoLab/CBIG/tree/master/stable_projects/brain_parcellation/Schaefer2018_LocalGlobal).

To prepare this atlas for the project, please perform the following operations:

1. Create a local folder for brain parcellations: (e.g., ```/data/SFIMJGC_Introspec/2023_fc_intrsopection/atlases```)


   > **NOTE**: Unless you are using a different version of the Schaefer Atlas, you can skip this step.


3. Create a sub-folder for the 200 Schaefer Atlas:
```bash
cd ${ATLASES_DIR}
mkdir Schaefer2018_200Parcels_7Networks
```

3. Copy the following files from their original location in CBIG repo (link above) to your local Schaefer Atlas folder:

* [```Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz```](https://github.com/ThomasYeoLab/CBIG/tree/master/stable_projects/brain_parcellation/Schaefer2018_LocalGlobal/Parcellations/MNI/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz): 200 ROI Atlas / 7 Yeo Networks in MNI space on a 2mm grid.

* [```Schaefer2018_200Parcels_7Networks_order.txt```](https://github.com/ThomasYeoLab/CBIG/tree/master/stable_projects/brain_parcellation/Schaefer2018_LocalGlobal/Parcellations/MNI/freeview_lut/Schaefer2018_200Parcels_7Networks_order.txt): list of all ROI, their network membership and color code (RGB).

* [```Schaefer2018_200Parcels_7Networks_order.lut```](https://github.com/ThomasYeoLab/CBIG/tree/master/stable_projects/brain_parcellation/Schaefer2018_LocalGlobal/Parcellations/MNI/fsleyes_lut/Schaefer2018_200Parcels_7Networks_order.lut): same as above, but coded for FSL eyes.

* [```Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv```](https://github.com/ThomasYeoLab/CBIG/tree/master/stable_projects/brain_parcellation/Schaefer2018_LocalGlobal/Parcellations/MNI/Centroid_coordinates/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv): information about each ROI centroid coordinates.

4. Correct space, generate label table and attach it to atlas file

In [2]:
# Correct the space tag, generate a label table, attach it to the original atlas file.
command = """module load afni; \
   cd {ATLAS_PATH}; \
   3drefit -space MNI {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz; \
   @MakeLabelTable -lab_file {ATLAS_NAME}_order.txt 1 0 -labeltable {ATLAS_NAME}_order.niml.lt -dset {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz;""".format(ATLAS_PATH=CORTICAL_ATLAS_PATH,ATLAS_NAME=CORTICAL_ATLAS_NAME)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
*+ WARNING: Changing the space of an ORIG view dataset may cause confusion!
*+ WARNING:  NIFTI copies will be interpreted as TLRC view (not TLRC space).
*+ WARNING:  Consider changing the view of the dataset to TLRC view also
 + loading and re-writing dataset Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets
++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
 + setting labeltable
 + loading and r

5. Convert Atlas file to final grid of all pre-processed data

In [3]:
# Put in the final grid that agress with that of all fully pre-processed functional scans
command = """module load afni; \
   cd {ATLAS_PATH}; \
   3dZeropad -overwrite -master {DATA_DIR}/PrcsData/ALL_SCANS/all_mean.box.nii.gz -prefix {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz; \
   3drefit -labeltable {ATLAS_NAME}_order.niml.lt {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz""".format(ATLAS_PATH=CORTICAL_ATLAS_PATH,ATLAS_NAME=CORTICAL_ATLAS_NAME, DATA_DIR=DATA_DIR)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3dZeropad: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ 3dZeropad -master => -I 0 -S -12 -A -6 -P -8 -R -7 -L -10
++ output dataset: ./Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
 + setting labeltable
 + loading and re-writing dataset Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets


6. Remove regions with bad coverage from the atlas

In [4]:
# Remove a series of regions with bad coverage / low signal levels
# Limbic LH Temporal Pole 1-4: 57,58,59,60
# Limbic RH Temporal Pole 1-3: 162,163,164
# Limbic LH OFC 1-2: 55,56
# Limbic RH OFC 1-3: 159,160,161
# LH_Default_Temp_2: 75 | for several scans this ROI caused issues 
command="""module load afni; \
           cd {ATLAS_PATH}; \
           3dcalc -overwrite \
                  -a {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz \
                  -expr 'equals(a,57)+equals(a,58)+equals(a,59)+equals(a,60)+equals(a,162)+equals(a,163)+equals(a,164)+equals(a,55)+equals(a,56)+equals(a,159)+equals(a,160)+equals(a,161)+equals(a,75)' \
                  -prefix {ATLAS_NAME}_Removed_ROIs.nii.gz; \
           3dcalc -overwrite \
                  -a      {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz \
                  -expr 'a-57*equals(a,57)-58*equals(a,58)-59*equals(a,59)-60*equals(a,60)-162*equals(a,162)-163*equals(a,163)-164*equals(a,164)-55*equals(a,55)-56*equals(a,56)-159*equals(a,159)-160*equals(a,160)-161*equals(a,161)-75*equals(a,75)' \
                  -prefix {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz; \
           3drefit -labeltable {ATLAS_NAME}_order.niml.lt {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz""".format(ATLAS_PATH=CORTICAL_ATLAS_PATH,ATLAS_NAME=CORTICAL_ATLAS_NAME, DATA_DIR=DATA_DIR)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3dcalc: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset ./Schaefer2018_200Parcels_7Networks_Removed_ROIs.nii.gz
++ 3dcalc: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset ./Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz
 + setting labeltable
 + loading and re-writing dataset Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets


7. Rank the atlas with missing ROIs

In [5]:
command = """module load afni; \
             cd {ATLAS_PATH}; \
             3dRank -prefix {ATLAS_NAME}_order_FSLMNI152_2mm.ranked.nii.gz -input {ATLAS_NAME}_order_FSLMNI152_2mm.nii.gz;""".format(ATLAS_PATH=CORTICAL_ATLAS_PATH,ATLAS_NAME=CORTICAL_ATLAS_NAME)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ Output dataset /gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.ranked.nii.gz


8. Create rank corrected Order & Centroid Files

In [6]:
path_to_order_file = osp.join(CORTICAL_ATLAS_PATH,'Schaefer2018_200Parcels_7Networks_order.txt')
path_to_rank_file  = osp.join(CORTICAL_ATLAS_PATH,'Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.ranked.nii.gz.rankmap.1D')
path_to_centroids_file = osp.join(CORTICAL_ATLAS_PATH,'Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv')
correct_ranked_atlas(path_to_order_file,path_to_centroids_file,path_to_rank_file)

++ INFO [correct_ranked_shaefer_atlas] Original Order File in memory [(200, 5)]
++ INFO [correct_ranked_shaefer_atlas] Original Centroids File in memory [(200, 4)]
++ INFO [correct_ranked_shaefer_atlas] Rank information in memory    [(188, 2)]
++ INFO [correct_ranked_shaefer_atlas] New order file written to disk: /data/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order.ranked.txt
++ INFO [correct_ranked_shaefer_atlas] New centroids file written to disk: /data/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.ranked.txt
 + Original Number of ROIs = 200
 + New      Number of ROIs = 187
 + Last entry in original Order File:
ID      7Networks_RH_Default_pCunPCC_3
R                                  208
G                                   63
B                                   77
Size                                 0
Name: 

9. Add corrected label table to the ranked version of the atlas

In [7]:
command = """module load afni; \
             cd {ATLAS_PATH}; \
             @MakeLabelTable -lab_file {ATLAS_NAME}_order.ranked.txt 1 0 -labeltable {ATLAS_NAME}_order.ranked.niml.lt -dset {ATLAS_NAME}_order_FSLMNI152_2mm.ranked.nii.gz;""".format(ATLAS_PATH=CORTICAL_ATLAS_PATH,ATLAS_NAME=CORTICAL_ATLAS_NAME)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.ranked.nii.gz
 + setting labeltable
 + loading and re-writing dataset Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.ranked.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.ranked.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets


10. Create a dataframe with all the necessary info about this atlas

We now create a single dataframe with all the info we need: roi number, roi label, network, hemisphere, colors codes and centroid position. We save this to disk so that it can be easily accessed by any other notebook

In [8]:
# Load the cetroid file for the ranked atlas in memory
centroids_info               = pd.read_csv(osp.join(ATLASES_DIR, CORTICAL_ATLAS_NAME,'{ATLAS_NAME}_order_FSLMNI152_2mm.Centroid_RAS.ranked.txt'.format(ATLAS_NAME=CORTICAL_ATLAS_NAME) ))
centroids_info['ROI Name']   = [label.split('7Networks_')[1] for label in centroids_info['ROI Name']]
centroids_info['Hemisphere'] = [item.split('_')[0] for item in centroids_info['ROI Name']]
centroids_info['Network']    = [item.split('_')[1] for item in centroids_info['ROI Name']]
# Load the color info file for the ranked atlas in memory
color_info = pd.read_csv(osp.join(ATLASES_DIR, CORTICAL_ATLAS_NAME,'{ATLAS_NAME}_order.ranked.txt'.format(ATLAS_NAME=CORTICAL_ATLAS_NAME)),sep='\t', header=None)
# Combine all the useful columns into a single new dataframe
df         = pd.concat([centroids_info[['ROI Label','Hemisphere','Network','ROI Name','R','A','S']],color_info[[2,3,4]]], axis=1)
df.columns = ['ROI_ID','Hemisphere','Network','ROI_Name','pos_R','pos_A','pos_S','color_R','color_G','color_B']
# Save the new data frame to disk
df.to_csv(osp.join(ATLASES_DIR,CORTICAL_ATLAS_NAME,'{ATLAS_NAME}_order_FSLMNI152_2mm.ranked.roi_info.csv'.format(ATLAS_NAME=CORTICAL_ATLAS_NAME)), index=False)

***
# 2. Prepare the AAL v2 Atlas

We will use this atlas to obtain defintions for 8 subcortical regions not includes in the Schaeffer atlas, namely L/R caudate, L/R putamen, L/R pallidum and L/R thalamus.

To prepare this atlas for the project, please preform the following operations:

1. Create a sub-folder within ```ATLASES_DIR``` for this second atlas

```bash
cd ${ATLASES_DIR}
mkdir aal2
```

2. Download the AAL v2 Atlas from [here](https://www.gin.cnrs.fr/en/tools/aal/)

3. Unzip the contents of the downloaded file into the AAL2 folder.

   > **NOTE**: we only need two files ```aal2.nii.gz``` and ```aal2.nii.txt```. So make sure those are available in ```${ATLASES_DIR}/aal2``` 

4. We need to correct the space in ```aal2.nii.gz``` to have the correct MNI space tag for AFNI

In [9]:
# Correct the space tag, generate a label table, attach it to the original atlas file.
command = """module load afni; \
   cd {ATLAS_PATH}; \
   3drefit -space MNI {ATLAS_NAME}.nii.gz; \
   @MakeLabelTable -lab_file {ATLAS_NAME}.nii.txt 1 0 -labeltable {ATLAS_NAME}.niml.lt -dset {ATLAS_NAME}.nii.gz;""".format(ATLAS_PATH=SUBCORTICAL_ATLAS_PATH,ATLAS_NAME=SUBCORTICAL_ATLAS_NAME)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset aal2.nii.gz
 + loading and re-writing dataset aal2.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2/aal2.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets
++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset aal2.nii.gz
 + setting labeltable
 + loading and re-writing dataset aal2.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2/aal2.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets


5. Convert Atlas to the same grid as the functional data

In [10]:
# Put in the final grid that agress with that of all fully pre-processed functional scans
command = """module load afni; \
   cd {ATLAS_PATH}; \
   3dZeropad -overwrite -master {DATA_DIR}/PrcsData/ALL_SCANS/all_mean.box.nii.gz -prefix {ATLAS_NAME}.nii.gz {ATLAS_NAME}.nii.gz; \
   3drefit -labeltable {ATLAS_NAME}.niml.lt {ATLAS_NAME}.nii.gz""".format(ATLAS_PATH=SUBCORTICAL_ATLAS_PATH,ATLAS_NAME=SUBCORTICAL_ATLAS_NAME, DATA_DIR=DATA_DIR)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3dZeropad: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ 3dZeropad -master => -I 0 -S 0 -A 0 -P 0 -R 0 -L 0
++ THD_zeropad: all pad values are zero - just copying dataset
++ output dataset: ./aal2.nii.gz
++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset aal2.nii.gz
 + setting labeltable
 + loading and re-writing dataset aal2.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2/aal2.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets


6. Create new atlas file with only the 8 ROIs we need

In [11]:
command = """module load afni; \
             cd {ATLAS_PATH}; \
             3dcalc -overwrite -a {ATLAS_NAME}.nii.gz \
                    -expr '75*equals(a,75) + 76*equals(a,76) + 77*equals(a,77) + 78*equals(a,78) + 79*equals(a,79) + 80*equals(a,80) + 81 * equals(a,81) + 82*equals(a,82)' \
                    -prefix {ATLAS_NAME}.subcortical.nii.gz; \
             3dmask_tool -overwrite -inputs {ATLAS_NAME}.subcortical.nii.gz -dilate_inputs -1 -prefix rm.{ATLAS_NAME}.subcortical.nii.gz; \
             3dcalc -overwrite -a {ATLAS_NAME}.subcortical.nii.gz -b rm.{ATLAS_NAME}.subcortical.nii.gz -expr 'a*b' -prefix {ATLAS_NAME}.subcortical.nii.gz; \
             3drefit -labeltable {ATLAS_NAME}.niml.lt {ATLAS_NAME}.subcortical.nii.gz; \
             rm rm.{ATLAS_NAME}.subcortical.nii.gz;""".format(ATLAS_PATH=SUBCORTICAL_ATLAS_PATH,ATLAS_NAME=SUBCORTICAL_ATLAS_NAME)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3dcalc: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset ./aal2.subcortical.nii.gz
++ no -frac option: defaulting to -union
++ processing 1 input dataset(s), NN=2...
++ padding all datasets by 0 (for dilations)
++ frac 0 over 1 volumes gives min count 0
++ voxel limits: 0 clipped, 2605 survived, 552765 were zero
++ writing result rm.aal2.subcortical.nii.gz...
++ Output dataset ./rm.aal2.subcortical.nii.gz
++ 3dcalc: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset ./aal2.subcortical.nii.gz
++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset aal2.subcortical.nii.gz
 + setting labeltable
 + loading and re-writing dataset aal2.subcortical.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2/aal2.subcortical.nii.g

7. Remove areas of overlap between the two atlases

Upon visual inspection we realized that despite having eroded the subcortical ROIs from the AAL2 atlas, we could still observe one voxel that overlaps with the Schaeffer atlas. Given at a later stage we will be merging both atlases, it is best to remove this voxel from the ```aal2.subcortical.nii.gz``` file now.

In [12]:
command="""module load afni; \
           cd {SUBCORTICAL_ATLAS_PATH};  \
           3dcalc -overwrite -a {SUBCORTICAL_ATLAS_NAME}.subcortical.nii.gz -b {CORTICAL_ATLAS_PATH}/{CORTICAL_ATLAS_NAME}_order_FSLMNI152_2mm.ranked.nii.gz -expr 'equals(0,step(a*b))' -prefix rm.overlap_between_atlases.nii.gz; \
           3dcalc -overwrite -a {SUBCORTICAL_ATLAS_NAME}.subcortical.nii.gz -b rm.overlap_between_atlases.nii.gz -expr 'a*b' -prefix {SUBCORTICAL_ATLAS_NAME}.subcortical.nii.gz; \
           3drefit -labeltable {SUBCORTICAL_ATLAS_NAME}.niml.lt {SUBCORTICAL_ATLAS_NAME}.subcortical.nii.gz; \
           rm rm.overlap_between_atlases.nii.gz;""".format(SUBCORTICAL_ATLAS_PATH=SUBCORTICAL_ATLAS_PATH, SUBCORTICAL_ATLAS_NAME=SUBCORTICAL_ATLAS_NAME, CORTICAL_ATLAS_NAME=CORTICAL_ATLAS_NAME, CORTICAL_ATLAS_PATH=CORTICAL_ATLAS_PATH)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3dcalc: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset ./rm.overlap_between_atlases.nii.gz
++ 3dcalc: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset ./aal2.subcortical.nii.gz
++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset aal2.subcortical.nii.gz
 + setting labeltable
 + loading and re-writing dataset aal2.subcortical.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2/aal2.subcortical.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets


8. Rank the atlas with missing ROIs

In [13]:
command = """module load afni; \
             cd {ATLAS_PATH}; \
             3dRank -prefix {ATLAS_NAME}.subcortical.ranked.nii.gz -input {ATLAS_NAME}.subcortical.nii.gz;""".format(ATLAS_PATH=SUBCORTICAL_ATLAS_PATH,ATLAS_NAME=SUBCORTICAL_ATLAS_NAME)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ Output dataset /gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2/aal2.subcortical.ranked.nii.gz


9. Create two additional text files with info about centroids and colors that we will need later

* ```aal2.subcortical_order.txt```: contains ROI_ID, ROI_Name, RGB Color, Size

In [14]:
command = """cd {ATLAS_PATH}; \
             cat {ATLAS_NAME}.nii.txt | grep -e Thalamus -e Pallidum -e Caudate -e Putamen | awk -F '[ _]' '{{print $1"\taal2_"$3"H_"$2"_1\t128\t128\t128\t0"}}' > {ATLAS_NAME}.subcortical_order.txt;
             """.format(ATLAS_PATH=SUBCORTICAL_ATLAS_PATH,ATLAS_NAME=SUBCORTICAL_ATLAS_NAME)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

* ```aal2.subcortical.Centroid_RAS.csv```: contains ROI_ID, ROI_Name , Centroid coordinates

```bash
cd /data/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2
module load afni
cat aal2.subcortical_order.txt | awk -F '\t' '{print $1","$2","}' > rm01.aal2.subcortical.Centroid_RAS.csv
3dCM -Icent -all_rois -Icent aal2.subcortical.nii.gz | grep -v '#' | tail -n 8 | awk '{print int($1)*-1","int($2)*-1","int($3)*-1}' > rm02.aal2.subcortical.Centroid_RAS.csv
paste -d'\0' rm01.aal2.subcortical.Centroid_RAS.csv rm02.aal2.subcortical.Centroid_RAS.csv > aal2.subcortical.Centroid_RAS.csv
sed -i '1s/^/ROI Label,ROI Name,R,A,S\n/' aal2.subcortical.Centroid_RAS.csv;
rm rm01.aal2.subcortical.Centroid_RAS.csv rm02.aal2.subcortical.Centroid_RAS.csv
```

10. Create Rank corrected Order and Centroid Files

In [15]:
path_to_order_file = osp.join(SUBCORTICAL_ATLAS_PATH,'aal2.subcortical_order.txt')
path_to_rank_file  = osp.join(SUBCORTICAL_ATLAS_PATH,'aal2.subcortical.ranked.nii.gz.rankmap.1D')
path_to_centroids_file = osp.join(SUBCORTICAL_ATLAS_PATH,'aal2.subcortical.Centroid_RAS.csv')
correct_ranked_atlas(path_to_order_file,path_to_centroids_file,path_to_rank_file)

++ INFO [correct_ranked_shaefer_atlas] Original Order File in memory [(8, 5)]
++ INFO [correct_ranked_shaefer_atlas] Original Centroids File in memory [(8, 4)]
++ INFO [correct_ranked_shaefer_atlas] Rank information in memory    [(9, 2)]
++ INFO [correct_ranked_shaefer_atlas] New order file written to disk: /data/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2/aal2.subcortical_order.ranked.txt
++ INFO [correct_ranked_shaefer_atlas] New centroids file written to disk: /data/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2/aal2.subcortical.Centroid_RAS.ranked.txt
 + Original Number of ROIs = 82
 + New      Number of ROIs = 8
 + Last entry in original Order File:
ID      aal2_RH_Thalamus_1
R                      128
G                      128
B                      128
Size                     0
Name: 82, dtype: object
 + Last entry in new Order File:
Number                     8
ID        aal2_RH_Thalamus_1
R                        128
G                        128
B             

11. Add corrected label table to the ranked version of the atlas

In [16]:
command = """module load afni; \
             cd {ATLAS_PATH}; \
             @MakeLabelTable -lab_file aal2.subcortical_order.ranked.txt 1 0 -labeltable aal2.subcortical_order.ranked.niml.lt -dset aal2.subcortical.ranked.nii.gz;""".format(ATLAS_PATH=SUBCORTICAL_ATLAS_PATH)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset aal2.subcortical.ranked.nii.gz
 + setting labeltable
 + loading and re-writing dataset aal2.subcortical.ranked.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/aal2/aal2.subcortical.ranked.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets


10. Create a dataframe with all the necessary info about this atlas

We now create a single dataframe with all the info we need: roi number, roi label, network, hemisphere, colors codes and centroid position. We save this to disk so that it can be easily accessed by any other notebook

In [17]:
# Load the cetroid file for the ranked atlas in memory
centroids_info               = pd.read_csv(osp.join(ATLASES_DIR, SUBCORTICAL_ATLAS_NAME,'aal2.subcortical.Centroid_RAS.ranked.txt' ))
centroids_info['ROI Name']   = [label.split('aal2_')[1] for label in centroids_info['ROI Name']]
centroids_info['Hemisphere'] = [item.split('_')[0] for item in centroids_info['ROI Name']]
centroids_info['Network']    = [item.split('_')[1] for item in centroids_info['ROI Name']]
# Load the color info file for the ranked atlas in memory
color_info = pd.read_csv(osp.join(ATLASES_DIR, SUBCORTICAL_ATLAS_NAME,'aal2.subcortical_order.ranked.txt'),sep='\t', header=None)
# Combine all the useful columns into a single new dataframe
df         = pd.concat([centroids_info[['ROI Label','Hemisphere','Network','ROI Name','R','A','S']],color_info[[2,3,4]]], axis=1)
df.columns = ['ROI_ID','Hemisphere','Network','ROI_Name','pos_R','pos_A','pos_S','color_R','color_G','color_B']
# Save the new data frame to disk
df.to_csv(osp.join(ATLASES_DIR,SUBCORTICAL_ATLAS_NAME,'aal2.subcortical.ranked.roi_info.csv'), index=False)

***
# 3. Combine Cortical and Subcortical Atlas

1. Create new folder for the combined atlas

In [29]:
if osp.exists(FB_ATLAS_PATH):
    rmtree(FB_ATLAS_PATH)
    print('++ WARNING: Removing pre-existing folder for combined atlas [%s]' % FB_ATLAS_PATH)
os.makedirs(FB_ATLAS_PATH)

++ WARNING: Removing pre-existing folder for combined atlas [/data/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks_AAL2]


2. Create un-ranked combined atlas NII file
```bash
cd /data/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks_AAL2
3dcalc -overwrite -a ../aal2/aal2.subcortical.ranked.nii.gz -expr 'step(a)*(200+a)' -prefix rm.subcortical.nii.gz
3dcalc -overwrite -a ../Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.ranked.nii.gz -b rm.subcortical.nii.gz -expr 'a+b' -prefix Schaefer2018_200Parcels_7Networks_AAL2.nii.gz
rm rm.subcortical.nii.gz
```

3. Create un-ranked combined atlas Order file
```bash
cat ../Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order.ranked.txt | awk '{gsub("7Networks","8Networks",$2); print $1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6}' > Schaefer2018_200Parcels_7Networks_AAL2_order.txt
cat ../aal2/aal2.subcortical_order.ranked.txt | awk '{gsub("aal2","8Networks",$2); print 200+$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6}' >> Schaefer2018_200Parcels_7Networks_AAL2_order.txt
```

4. Create un-ranked combined atlas Centroid file
```bash
head -n 1 ../Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.ranked.txt                                                                                > Schaefer2018_200Parcels_7Networks_AAL2.Centroid_RAS.txt
sed '1d' ../Schaefer2018_200Parcels_7Networks/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.ranked.txt | awk -F ',' '{gsub("7Networks","8Networks",$2); print $0}' | tr -s ' ' ',' >> Schaefer2018_200Parcels_7Networks_AAL2.Centroid_RAS.txt
tail -n 8 ../aal2/aal2.subcortical.Centroid_RAS.ranked.txt | awk -F ',' '{gsub("aal2","8Networks",$2); print 200+$1","$2","$3","$4","$5}'                                                                  >> Schaefer2018_200Parcels_7Networks_AAL2.Centroid_RAS.txt
```

5. Rank the atlas with missing ROIs

In [30]:
command = """module load afni; \
             cd {ATLAS_PATH}; \
             3dRank -prefix {ATLAS_NAME}.ranked.nii.gz -input {ATLAS_NAME}.nii.gz;""".format(ATLAS_PATH=FB_ATLAS_PATH,ATLAS_NAME=FB_ATLAS_NAME)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ Output dataset /gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks_AAL2/Schaefer2018_200Parcels_7Networks_AAL2.ranked.nii.gz


6. Create Rank corrected Order and Centroid Files

In [32]:
path_to_order_file = osp.join(FB_ATLAS_PATH,'{ATLAS_NAME}_order.txt'.format(ATLAS_NAME=FB_ATLAS_NAME))
path_to_rank_file  = osp.join(FB_ATLAS_PATH,'{ATLAS_NAME}.ranked.nii.gz.rankmap.1D'.format(ATLAS_NAME=FB_ATLAS_NAME))
path_to_centroids_file = osp.join(FB_ATLAS_PATH,'{ATLAS_NAME}.Centroid_RAS.txt'.format(ATLAS_NAME=FB_ATLAS_NAME))
correct_ranked_atlas(path_to_order_file,path_to_centroids_file,path_to_rank_file)

++ INFO [correct_ranked_shaefer_atlas] Original Order File in memory [(195, 5)]
++ INFO [correct_ranked_shaefer_atlas] Original Centroids File in memory [(195, 4)]
++ INFO [correct_ranked_shaefer_atlas] Rank information in memory    [(196, 2)]
++ INFO [correct_ranked_shaefer_atlas] New order file written to disk: /data/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks_AAL2/Schaefer2018_200Parcels_7Networks_AAL2_order.ranked.txt
++ INFO [correct_ranked_shaefer_atlas] New centroids file written to disk: /data/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks_AAL2/Schaefer2018_200Parcels_7Networks_AAL2.Centroid_RAS.ranked.txt
 + Original Number of ROIs = 208
 + New      Number of ROIs = 195
 + Last entry in original Order File:
ID      8Networks_RH_Thalamus_1
R                           128
G                           128
B                           128
Size                          0
Name: 208, dtype: object
 + Last entry in

11. Add corrected label table to the ranked version of the atlas

In [33]:
command = """module load afni; \
             cd {ATLAS_PATH}; \
             @MakeLabelTable -lab_file {ATLAS_NAME}_order.ranked.txt 1 0 -labeltable {ATLAS_NAME}_order.ranked.niml.lt -dset {ATLAS_NAME}.ranked.nii.gz;
             """.format(ATLAS_PATH=FB_ATLAS_PATH, ATLAS_NAME=FB_ATLAS_NAME)
output  = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
print(output.strip().decode())

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated  2023-02-10

++ 3drefit: AFNI version=AFNI_23.0.03 (Feb  6 2023) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset Schaefer2018_200Parcels_7Networks_AAL2.ranked.nii.gz
 + setting labeltable
 + loading and re-writing dataset Schaefer2018_200Parcels_7Networks_AAL2.ranked.nii.gz (/gpfs/gsfs11/users/SFIMJGC_Introspec/2023_fc_introspection/atlases/Schaefer2018_200Parcels_7Networks_AAL2/Schaefer2018_200Parcels_7Networks_AAL2.ranked.nii.gz in NIFTI storage)
++ 3drefit processed 1 datasets


10. Create a dataframe with all the necessary info about this atlas

We now create a single dataframe with all the info we need: roi number, roi label, network, hemisphere, colors codes and centroid position. We save this to disk so that it can be easily accessed by any other notebook

In [35]:
# Load the cetroid file for the ranked atlas in memory
centroids_info               = pd.read_csv(osp.join(ATLASES_DIR, FB_ATLAS_NAME,'{ATLAS_NAME}.Centroid_RAS.ranked.txt'.format(ATLAS_NAME=FB_ATLAS_NAME) ))
centroids_info['ROI Name']   = [label.split('8Networks_')[1] for label in centroids_info['ROI Name']]
centroids_info['Hemisphere'] = [item.split('_')[0] for item in centroids_info['ROI Name']]
centroids_info['Network']    = [item.split('_')[1] for item in centroids_info['ROI Name']]
# Load the color info file for the ranked atlas in memory
color_info = pd.read_csv(osp.join(ATLASES_DIR, FB_ATLAS_NAME,'{ATLAS_NAME}_order.ranked.txt'.format(ATLAS_NAME=FB_ATLAS_NAME)),sep='\t', header=None)
# Combine all the useful columns into a single new dataframe
df         = pd.concat([centroids_info[['ROI Label','Hemisphere','Network','ROI Name','R','A','S']],color_info[[2,3,4]]], axis=1)
df.columns = ['ROI_ID','Hemisphere','Network','ROI_Name','pos_R','pos_A','pos_S','color_R','color_G','color_B']
# Save the new data frame to disk
df.to_csv(osp.join(ATLASES_DIR,FB_ATLAS_NAME,'{ATLAS_NAME}.ranked.roi_info.csv'.format(ATLAS_NAME=FB_ATLAS_NAME)), index=False)